In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import time, h5py
import xgboost as xgb
import lightgbm as lgb
import category_encoders as cat_ed
import gc, mlcrate, shap

from fastai.imports import *
from fastai.structured import *
from gplearn.genetic import SymbolicTransformer
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from catboost import CatBoostClassifier
from collections import Counter
from scipy.cluster import hierarchy as hc
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import  roc_auc_score, log_loss
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from collections import Counter

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Reshape
from keras.layers import Merge
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint

# will ignore all warning from sklearn, seaborn etc..
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

pd.option_context("display.max_rows", 1000);
pd.option_context("display.max_columns", 250);

In [3]:
PATH = os.getcwd();
PATH

'D:\\Github\\fastai\\courses\\ml1'

In [4]:
df_raw  = pd.read_csv(f'{PATH}\\AV_Stud\\models\\train_140618.csv', low_memory= False)
df_test = pd.read_csv(f'{PATH}\\AV_Stud\\models\\test_140618.csv', low_memory=False)

In [5]:
target = df_raw.is_pass.values
df_raw.drop(['is_pass'],axis=1, inplace=True)

features = df_raw.columns
numeric_features = []
categorical_features = []

for dtype, feature in zip(df_raw.dtypes, df_raw.columns):
    if dtype == object:
        categorical_features.append(feature)
    else:
        numeric_features.append(feature)
        
categorical_features;

X_train_cats = df_raw[categorical_features] 
X_test_cats  = df_test[categorical_features]

In [6]:
X_train_dnn = X_train_cats.values
X_test_dnn = X_test_cats.values

idx = X_train_cats.shape[0]; idx

73147

In [7]:
data = pd.concat([X_train_cats, X_test_cats], axis = 0)
data.shape, idx

((104496, 14), 73147)

In [8]:
features_idx = [i for i in range(14)];
features_idx

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [9]:
les = []
for i in range(X_train_dnn.shape[1]):
    le = LabelEncoder()
    le.fit(data.iloc[:,features_idx].iloc[:, i])
    les.append(le)
    X_train_dnn[:, i] = le.transform(X_train_dnn[:, i])
    X_test_dnn[:, i]  = le.transform(X_test_dnn[:, i])   

In [10]:
def split_features(X):
    
    X_list = []
    
    C1 = X[..., [0]]
    X_list.append(C1)
    
    program_type = X[..., [1]]
    X_list.append(program_type)
    
    test_type = X[..., [2]]
    X_list.append(test_type)
    
    difficulty_level = X[..., [3]]
    X_list.append(difficulty_level)
    
    gender = X[..., [4]]
    X_list.append(gender)
    
    education = X[..., [5]]
    X_list.append(education)
    
    is_handicapped = X[..., [6]]
    X_list.append(is_handicapped)
    
    program_type__program_duration = X[..., [7]]
    X_list.append(program_type__program_duration)
    
    test_id__program_duration = X[..., [8]]
    X_list.append(test_id__program_duration)
    
    test_id__test_type = X[..., [9]]
    X_list.append(test_id__test_type)
    
    test_type__difficulty_level = X[..., [10]]
    X_list.append(test_type__difficulty_level)
    
    education__gender = X[..., [11]]
    X_list.append(education__gender)
    
    C14 = X[..., [12]]#education__city_tier
    X_list.append(C14)
    
    C15 = X[..., [13]]#gender__city_tier
    X_list.append(C15)
    
    return X_list

In [11]:
for i in X_train_cats.columns:
    print(i, len(np.unique(df_raw[i].values)))

program_id 22
program_type 7
test_type 2
difficulty_level 4
gender 2
education 5
is_handicapped 2
program_type__program_duration 22
test_id__program_duration 188
test_id__test_type 188
test_type__difficulty_level 5
education__gender 10
education__city_tier 19
gender__city_tier 8


In [12]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train_dnn, target, train_size=0.8, random_state=1234)

In [16]:
class NN_with_EntityEmbedding(object):
    
    def __init__(self, X_train, y_train, X_val, y_val):
        self.nb_epoch = 10
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)
        
    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list
    
    def __build_keras_model(self):
        
        '''
        program_id 22
        program_type 7
        test_type 2
        difficulty_level 4
        gender 2
        education 5
        is_handicapped 2
        program_type__program_duration 22
        test_id__program_duration 188
        test_id__test_type 188
        test_type__difficulty_level 5
        education__gender 10
        education__city_tier 19
        gender__city_tier 8
       '''
        models = []
        model_C1= Sequential() #program_id
        model_C1.add(Embedding(len(les[0].classes_), 22, input_length=1))
        model_C1.add(Reshape(target_shape=(22,)))
        models.append(model_C1)
        
        model_program_type = Sequential()
        model_program_type.add(Embedding(len(les[1].classes_), 7, input_length=1))
        model_program_type.add(Reshape(target_shape=(7,)))
        models.append(model_program_type)
        
        model_test_type = Sequential()
        model_test_type.add(Embedding(len(les[2].classes_), 3, input_length=1))
        model_test_type.add(Reshape(target_shape=(3,)))
        models.append(model_test_type)
        
        difficulty_level = Sequential()
        difficulty_level.add(Embedding(len(les[3].classes_), 5, input_length=1))
        difficulty_level.add(Reshape(target_shape=(5,)))
        models.append(difficulty_level)
        
        gender = Sequential()
        gender.add(Embedding(len(les[4].classes_), 3, input_length=1))
        gender.add(Reshape(target_shape=(3,)))
        models.append(gender)
        
        education = Sequential()
        education.add(Embedding(len(les[5].classes_), 6, input_length=1))
        education.add(Reshape(target_shape=(6,)))
        models.append(education)
        
        is_handicapped = Sequential()
        is_handicapped.add(Embedding(len(les[6].classes_), 3, input_length=1))
        is_handicapped.add(Reshape(target_shape=(3,)))
        models.append(is_handicapped)
        
        program_type__program_duration = Sequential()
        program_type__program_duration.add(Embedding(len(les[7].classes_), 23, input_length=1))
        program_type__program_duration.add(Reshape(target_shape=(23,)))
        models.append(program_type__program_duration)
        
        test_id__program_duration = Sequential()
        test_id__program_duration.add(Embedding(len(les[8].classes_), 30, input_length=1))
        test_id__program_duration.add(Reshape(target_shape=(30,)))
        models.append(test_id__program_duration)
        
        test_id__test_type = Sequential()
        test_id__test_type.add(Embedding(len(les[9].classes_), 30, input_length=1))
        test_id__test_type.add(Reshape(target_shape=(30,)))
        models.append(test_id__test_type)
        
        test_type__difficulty_level = Sequential()
        test_type__difficulty_level.add(Embedding(len(les[10].classes_), 6, input_length=1))
        test_type__difficulty_level.add(Reshape(target_shape=(6,)))
        models.append(test_type__difficulty_level)
        
        education__gender = Sequential()
        education__gender.add(Embedding(len(les[11].classes_), 10, input_length=1))
        education__gender.add(Reshape(target_shape=(10,)))
        models.append(education__gender)
        
        C14 = Sequential()#education__city_tier'
        C14.add(Embedding(len(les[12].classes_), 20, input_length=1))
        C14.add(Reshape(target_shape=(20,)))
        models.append(C14)
        
        C15 = Sequential()#gender__city_tier
        C15.add(Embedding(len(les[13].classes_), 9, input_length=1))
        C15.add(Reshape(target_shape=(9,)))
        models.append(C15)
        
        self.model = Sequential()
        self.model.add(Merge(models, mode='concat'))
        self.model.add(Dense(128, kernel_initializer='uniform'))
        self.model.add(Activation('relu'))
        
        self.model.add(Dense(256, kernel_initializer='uniform'))
        self.model.add(Activation('relu'))
        
        self.model.add(Dense(64, kernel_initializer='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))
        self.model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
        
    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocessing(X_train), y_train,validation_data=(self.preprocessing(X_val), y_val),\
                       epochs=self.nb_epoch, batch_size=64)

In [17]:
#X_train, X_validation, y_train, y_validation
dnn = NN_with_EntityEmbedding(X_train, y_train, X_validation, y_validation)
weights = dnn.model.get_weights()

Train on 58517 samples, validate on 14630 samples
Epoch 1/10


52096/58517 [=========================>....] - ETA: 633s - loss: 0.6933 - acc: 0.421 - ETA: 137s - loss: 0.6906 - acc: 0.621 - ETA: 81s - loss: 0.6841 - acc: 0.651 - ETA: 59s - loss: 0.6743 - acc: 0.67 - ETA: 48s - loss: 0.6595 - acc: 0.67 - ETA: 41s - loss: 0.6587 - acc: 0.67 - ETA: 36s - loss: 0.6403 - acc: 0.68 - ETA: 33s - loss: 0.6359 - acc: 0.68 - ETA: 30s - loss: 0.6288 - acc: 0.68 - ETA: 28s - loss: 0.6195 - acc: 0.69 - ETA: 26s - loss: 0.6208 - acc: 0.68 - ETA: 25s - loss: 0.6192 - acc: 0.69 - ETA: 23s - loss: 0.6163 - acc: 0.69 - ETA: 22s - loss: 0.6155 - acc: 0.68 - ETA: 21s - loss: 0.6128 - acc: 0.69 - ETA: 20s - loss: 0.6099 - acc: 0.69 - ETA: 20s - loss: 0.6105 - acc: 0.69 - ETA: 19s - loss: 0.6067 - acc: 0.69 - ETA: 19s - loss: 0.6058 - acc: 0.69 - ETA: 18s - loss: 0.6052 - acc: 0.69 - ETA: 18s - loss: 0.6021 - acc: 0.69 - ETA: 17s - loss: 0.6030 - acc: 0.69 - ETA: 17s - loss: 0.6023 - acc: 0.69 - ETA: 17s - loss: 0.6036 - acc: 0.68 - ETA: 16s - loss: 0.6036 - acc: 0.68 

58517/58517 [==============================] - ETA: 1s - loss: 0.5707 - acc: 0.701 - ETA: 1s - loss: 0.5704 - acc: 0.701 - ETA: 1s - loss: 0.5703 - acc: 0.701 - ETA: 1s - loss: 0.5702 - acc: 0.701 - ETA: 1s - loss: 0.5703 - acc: 0.701 - ETA: 1s - loss: 0.5704 - acc: 0.701 - ETA: 1s - loss: 0.5705 - acc: 0.701 - ETA: 1s - loss: 0.5703 - acc: 0.701 - ETA: 1s - loss: 0.5703 - acc: 0.701 - ETA: 0s - loss: 0.5701 - acc: 0.702 - ETA: 0s - loss: 0.5700 - acc: 0.702 - ETA: 0s - loss: 0.5703 - acc: 0.701 - ETA: 0s - loss: 0.5703 - acc: 0.701 - ETA: 0s - loss: 0.5703 - acc: 0.701 - ETA: 0s - loss: 0.5703 - acc: 0.701 - ETA: 0s - loss: 0.5703 - acc: 0.701 - ETA: 0s - loss: 0.5704 - acc: 0.701 - ETA: 0s - loss: 0.5704 - acc: 0.702 - ETA: 0s - loss: 0.5704 - acc: 0.702 - ETA: 0s - loss: 0.5704 - acc: 0.702 - ETA: 0s - loss: 0.5702 - acc: 0.702 - ETA: 0s - loss: 0.5701 - acc: 0.702 - ETA: 0s - loss: 0.5702 - acc: 0.702 - ETA: 0s - loss: 0.5701 - acc: 0.702 - ETA: 0s - loss: 0.5701 - acc: 0.702 - ETA

50496/58517 [========================>.....] - ETA: 14s - loss: 0.6210 - acc: 0.64 - ETA: 13s - loss: 0.5645 - acc: 0.71 - ETA: 14s - loss: 0.5589 - acc: 0.72 - ETA: 14s - loss: 0.5600 - acc: 0.71 - ETA: 14s - loss: 0.5598 - acc: 0.71 - ETA: 14s - loss: 0.5631 - acc: 0.71 - ETA: 14s - loss: 0.5637 - acc: 0.71 - ETA: 14s - loss: 0.5628 - acc: 0.71 - ETA: 14s - loss: 0.5645 - acc: 0.71 - ETA: 14s - loss: 0.5713 - acc: 0.70 - ETA: 14s - loss: 0.5734 - acc: 0.70 - ETA: 13s - loss: 0.5705 - acc: 0.70 - ETA: 13s - loss: 0.5737 - acc: 0.70 - ETA: 13s - loss: 0.5720 - acc: 0.70 - ETA: 13s - loss: 0.5669 - acc: 0.71 - ETA: 13s - loss: 0.5668 - acc: 0.71 - ETA: 13s - loss: 0.5658 - acc: 0.71 - ETA: 13s - loss: 0.5656 - acc: 0.71 - ETA: 13s - loss: 0.5656 - acc: 0.71 - ETA: 13s - loss: 0.5653 - acc: 0.70 - ETA: 13s - loss: 0.5649 - acc: 0.71 - ETA: 13s - loss: 0.5630 - acc: 0.71 - ETA: 13s - loss: 0.5630 - acc: 0.71 - ETA: 13s - loss: 0.5632 - acc: 0.70 - ETA: 13s - loss: 0.5630 - acc: 0.71 - ETA

58517/58517 [==============================] - ETA: 1s - loss: 0.5613 - acc: 0.708 - ETA: 1s - loss: 0.5615 - acc: 0.708 - ETA: 1s - loss: 0.5615 - acc: 0.708 - ETA: 1s - loss: 0.5615 - acc: 0.708 - ETA: 1s - loss: 0.5617 - acc: 0.708 - ETA: 1s - loss: 0.5618 - acc: 0.708 - ETA: 1s - loss: 0.5617 - acc: 0.708 - ETA: 1s - loss: 0.5616 - acc: 0.708 - ETA: 1s - loss: 0.5617 - acc: 0.708 - ETA: 1s - loss: 0.5616 - acc: 0.708 - ETA: 1s - loss: 0.5616 - acc: 0.708 - ETA: 1s - loss: 0.5618 - acc: 0.708 - ETA: 1s - loss: 0.5617 - acc: 0.708 - ETA: 1s - loss: 0.5616 - acc: 0.708 - ETA: 1s - loss: 0.5614 - acc: 0.708 - ETA: 1s - loss: 0.5614 - acc: 0.708 - ETA: 0s - loss: 0.5611 - acc: 0.709 - ETA: 0s - loss: 0.5612 - acc: 0.708 - ETA: 0s - loss: 0.5612 - acc: 0.709 - ETA: 0s - loss: 0.5611 - acc: 0.709 - ETA: 0s - loss: 0.5613 - acc: 0.708 - ETA: 0s - loss: 0.5615 - acc: 0.708 - ETA: 0s - loss: 0.5615 - acc: 0.708 - ETA: 0s - loss: 0.5615 - acc: 0.708 - ETA: 0s - loss: 0.5614 - acc: 0.708 - ETA

...

58517/58517 [==============================] - ETA: 1s - loss: 0.5545 - acc: 0.715 - ETA: 1s - loss: 0.5549 - acc: 0.715 - ETA: 1s - loss: 0.5549 - acc: 0.714 - ETA: 1s - loss: 0.5551 - acc: 0.714 - ETA: 1s - loss: 0.5549 - acc: 0.714 - ETA: 1s - loss: 0.5550 - acc: 0.714 - ETA: 1s - loss: 0.5549 - acc: 0.715 - ETA: 1s - loss: 0.5550 - acc: 0.714 - ETA: 1s - loss: 0.5551 - acc: 0.714 - ETA: 1s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5552 - acc: 0.714 - ETA: 0s - loss: 0.5554 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5552 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5552 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5555 - acc: 0.714 - ETA: 0s - loss: 0.5554 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA: 0s - loss: 0.5553 - acc: 0.714 - ETA

47872/58517 [=======================>......] - ETA: 14s - loss: 0.5436 - acc: 0.71 - ETA: 14s - loss: 0.5489 - acc: 0.72 - ETA: 14s - loss: 0.5692 - acc: 0.70 - ETA: 15s - loss: 0.5731 - acc: 0.71 - ETA: 15s - loss: 0.5608 - acc: 0.71 - ETA: 14s - loss: 0.5527 - acc: 0.72 - ETA: 14s - loss: 0.5497 - acc: 0.72 - ETA: 14s - loss: 0.5457 - acc: 0.72 - ETA: 14s - loss: 0.5498 - acc: 0.72 - ETA: 13s - loss: 0.5524 - acc: 0.72 - ETA: 13s - loss: 0.5499 - acc: 0.71 - ETA: 13s - loss: 0.5540 - acc: 0.71 - ETA: 13s - loss: 0.5555 - acc: 0.71 - ETA: 13s - loss: 0.5549 - acc: 0.71 - ETA: 13s - loss: 0.5552 - acc: 0.71 - ETA: 13s - loss: 0.5549 - acc: 0.71 - ETA: 13s - loss: 0.5546 - acc: 0.71 - ETA: 13s - loss: 0.5547 - acc: 0.71 - ETA: 13s - loss: 0.5551 - acc: 0.71 - ETA: 13s - loss: 0.5548 - acc: 0.71 - ETA: 13s - loss: 0.5537 - acc: 0.71 - ETA: 13s - loss: 0.5551 - acc: 0.71 - ETA: 13s - loss: 0.5534 - acc: 0.71 - ETA: 12s - loss: 0.5523 - acc: 0.71 - ETA: 12s - loss: 0.5528 - acc: 0.71 - ETA

58517/58517 [==============================] - ETA: 2s - loss: 0.5534 - acc: 0.715 - ETA: 2s - loss: 0.5533 - acc: 0.715 - ETA: 2s - loss: 0.5531 - acc: 0.715 - ETA: 2s - loss: 0.5533 - acc: 0.715 - ETA: 2s - loss: 0.5534 - acc: 0.715 - ETA: 2s - loss: 0.5535 - acc: 0.715 - ETA: 2s - loss: 0.5537 - acc: 0.715 - ETA: 2s - loss: 0.5537 - acc: 0.714 - ETA: 2s - loss: 0.5538 - acc: 0.714 - ETA: 2s - loss: 0.5539 - acc: 0.714 - ETA: 2s - loss: 0.5538 - acc: 0.714 - ETA: 2s - loss: 0.5540 - acc: 0.714 - ETA: 2s - loss: 0.5540 - acc: 0.714 - ETA: 1s - loss: 0.5542 - acc: 0.714 - ETA: 1s - loss: 0.5542 - acc: 0.714 - ETA: 1s - loss: 0.5540 - acc: 0.714 - ETA: 1s - loss: 0.5539 - acc: 0.714 - ETA: 1s - loss: 0.5538 - acc: 0.714 - ETA: 1s - loss: 0.5539 - acc: 0.714 - ETA: 1s - loss: 0.5541 - acc: 0.714 - ETA: 1s - loss: 0.5538 - acc: 0.714 - ETA: 1s - loss: 0.5538 - acc: 0.714 - ETA: 1s - loss: 0.5541 - acc: 0.714 - ETA: 1s - loss: 0.5541 - acc: 0.714 - ETA: 1s - loss: 0.5540 - acc: 0.714 - ETA

49280/58517 [========================>.....] - ETA: 18s - loss: 0.5748 - acc: 0.68 - ETA: 14s - loss: 0.5465 - acc: 0.70 - ETA: 13s - loss: 0.5194 - acc: 0.73 - ETA: 14s - loss: 0.5256 - acc: 0.73 - ETA: 13s - loss: 0.5501 - acc: 0.71 - ETA: 13s - loss: 0.5517 - acc: 0.71 - ETA: 13s - loss: 0.5554 - acc: 0.71 - ETA: 13s - loss: 0.5573 - acc: 0.71 - ETA: 13s - loss: 0.5520 - acc: 0.71 - ETA: 13s - loss: 0.5486 - acc: 0.71 - ETA: 13s - loss: 0.5464 - acc: 0.71 - ETA: 13s - loss: 0.5463 - acc: 0.71 - ETA: 13s - loss: 0.5475 - acc: 0.71 - ETA: 13s - loss: 0.5542 - acc: 0.71 - ETA: 13s - loss: 0.5519 - acc: 0.71 - ETA: 13s - loss: 0.5496 - acc: 0.71 - ETA: 13s - loss: 0.5520 - acc: 0.71 - ETA: 13s - loss: 0.5528 - acc: 0.71 - ETA: 12s - loss: 0.5524 - acc: 0.71 - ETA: 12s - loss: 0.5522 - acc: 0.71 - ETA: 12s - loss: 0.5526 - acc: 0.71 - ETA: 12s - loss: 0.5512 - acc: 0.71 - ETA: 12s - loss: 0.5523 - acc: 0.71 - ETA: 12s - loss: 0.5526 - acc: 0.71 - ETA: 12s - loss: 0.5514 - acc: 0.71 - ETA

58517/58517 [==============================] - ETA: 2s - loss: 0.5542 - acc: 0.715 - ETA: 2s - loss: 0.5540 - acc: 0.715 - ETA: 2s - loss: 0.5540 - acc: 0.715 - ETA: 2s - loss: 0.5540 - acc: 0.715 - ETA: 2s - loss: 0.5540 - acc: 0.715 - ETA: 1s - loss: 0.5542 - acc: 0.715 - ETA: 1s - loss: 0.5541 - acc: 0.715 - ETA: 1s - loss: 0.5539 - acc: 0.715 - ETA: 1s - loss: 0.5539 - acc: 0.715 - ETA: 1s - loss: 0.5538 - acc: 0.715 - ETA: 1s - loss: 0.5538 - acc: 0.715 - ETA: 1s - loss: 0.5535 - acc: 0.715 - ETA: 1s - loss: 0.5536 - acc: 0.715 - ETA: 1s - loss: 0.5540 - acc: 0.715 - ETA: 1s - loss: 0.5538 - acc: 0.715 - ETA: 1s - loss: 0.5538 - acc: 0.715 - ETA: 1s - loss: 0.5539 - acc: 0.715 - ETA: 1s - loss: 0.5537 - acc: 0.715 - ETA: 1s - loss: 0.5539 - acc: 0.715 - ETA: 1s - loss: 0.5540 - acc: 0.715 - ETA: 1s - loss: 0.5539 - acc: 0.715 - ETA: 1s - loss: 0.5539 - acc: 0.715 - ETA: 0s - loss: 0.5540 - acc: 0.715 - ETA: 0s - loss: 0.5537 - acc: 0.715 - ETA: 0s - loss: 0.5538 - acc: 0.715 - ETA

In [18]:
n = 0
for i in range(0,28,2):
    n+=(weights[i][0].shape)[1]

In [19]:
x_dnn_train = np.random.random((len(X_train_dnn),n))
start_ind=0
for j in range(X_train_dnn.shape[1]):
    mat = weights[j*2][0]
    dim = mat.shape[1]
    for i in range(X_train_dnn.shape[0]):
        x_dnn_train[i,start_ind:start_ind+dim]=mat[X_train_dnn[i,j]]
    start_ind += dim

In [20]:
x_dnn_test = np.random.random((len(X_test_dnn),n))
start_ind=0
for j in range(X_test_dnn.shape[1]):
    mat = weights[j*2][0]
    dim = mat.shape[1]
    for i in range(x_dnn_test.shape[0]):
        x_dnn_test[i,start_ind:start_ind+dim]=mat[X_test_dnn[i,j]]
    start_ind += dim

In [21]:
from sklearn.linear_model import LogisticRegression
l = LogisticRegression(n_jobs = -1)
r = RandomForestClassifier(n_estimators=100, max_depth= 10, n_jobs=-1)

In [22]:
l.fit(x_dnn_train,target)
y_pred_lr = l.predict_proba(x_dnn_test)[:,1]

In [23]:
y_pred_lr.shape

(31349,)

In [24]:
r.fit(x_dnn_train,target)
y_pred_rf = r.predict_proba(x_dnn_test)[:,1]

In [25]:
def make_submission(probs):
    sample = pd.read_csv(f'{PATH}\\AV_Stud\\sample_submission_vaSxamm.csv')
    submit = sample.copy()
    submit['is_pass'] = probs
    return submit

In [26]:
submit = make_submission(y_pred_rf)

In [27]:
submit.to_csv(f'{PATH}\\AV_Stud\\rf_cat_embedding.csv', index=False)

In [28]:
params = {}
params['booster'] = 'gbtree'
#params['updater'] = 'coord_descent'
params["objective"] = "binary:logistic"
params['eval_metric'] = 'auc'
params["eta"] = 0.05 #0.03
params["subsample"] = .90 #.85 was tried before
params["silent"] = 0
params['verbose'] = 1
params["max_depth"] = 11
params["seed"] = 1
params["max_delta_step"] = 4
params['scale_pos_weight'] =  0.4380049934141978
params["gamma"] = 0.6 #.5 #.1 #.2
params['colsample_bytree'] = 0.9
params['nrounds'] = 2000 #3600 #2000 #4000 #using lower no for demo
#params['verbose_eval'] = 50

In [29]:
x_train_final_dnn = np.hstack((x_dnn_train, df_raw[numeric_features].values))
x_test_final_dnn = np.hstack((x_dnn_test, df_test[numeric_features].values))

In [33]:
model_xgb, p_train_xgb, p_test_xgb= mlcrate.xgb.train_kfold(params, x_train_final_dnn, target,x_test_final_dnn\
                                                       , folds = 7, stratify=target, print_imp='final')

[mlcrate] Training 7 stratified XGBoost models on training set (73147, 210) with test set (31349, 210)
[mlcrate] Running fold 0, 62697 train samples, 10450 validation samples
[0]	train-auc:0.801348	valid-auc:0.704596
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[1]	train-auc:0.82629	valid-auc:0.718576
[2]	train-auc:0.83605	valid-auc:0.722714
[3]	train-auc:0.842477	valid-auc:0.726288
[4]	train-auc:0.847346	valid-auc:0.728381
[5]	train-auc:0.850894	valid-auc:0.730341
[6]	train-auc:0.854576	valid-auc:0.731289
[7]	train-auc:0.856952	valid-auc:0.731565
[8]	train-auc:0.858826	valid-auc:0.73275
[9]	train-auc:0.861137	valid-auc:0.733226
[10]	train-auc:0.863073	valid-auc:0.734136
[11]	train-auc:0.86531	valid-auc:0.734426
[12]	train-auc:0.867175	valid-auc:0.734889
[13]	train-auc:0.869013	valid-auc:0.735064
[14]	train-auc:0.87097	valid-auc:0.735869
[15]	train-auc:0.873033	valid-auc:0.73573
[16]	train

[183]	train-auc:0.979561	valid-auc:0.755552
[184]	train-auc:0.97967	valid-auc:0.755608
[185]	train-auc:0.979803	valid-auc:0.755584
[186]	train-auc:0.979979	valid-auc:0.755716
[187]	train-auc:0.980592	valid-auc:0.755853
[188]	train-auc:0.980821	valid-auc:0.755879
[189]	train-auc:0.980901	valid-auc:0.755924
[190]	train-auc:0.981108	valid-auc:0.756051
[191]	train-auc:0.981554	valid-auc:0.756219
[192]	train-auc:0.981636	valid-auc:0.756225
[193]	train-auc:0.981744	valid-auc:0.756233
[194]	train-auc:0.982068	valid-auc:0.756224
[195]	train-auc:0.982333	valid-auc:0.756105
[196]	train-auc:0.982531	valid-auc:0.756179
[197]	train-auc:0.98266	valid-auc:0.756167
[198]	train-auc:0.982772	valid-auc:0.756097
[199]	train-auc:0.982984	valid-auc:0.75614
[200]	train-auc:0.983102	valid-auc:0.756094
[201]	train-auc:0.983334	valid-auc:0.756103
[202]	train-auc:0.983606	valid-auc:0.756142
[203]	train-auc:0.983742	valid-auc:0.756044
[204]	train-auc:0.98404	valid-auc:0.755904
[205]	train-auc:0.984332	valid-auc:0

C:\ProgramData\Anaconda3\lib\site-packages\mlcrate\backend.py:7: UserWarning: Timer.format_elapsed() has been deprecated in favour of Timer.fsince() and will be removed soon
  warn(message)


[mlcrate] Finished training fold 0 - took 3m18s - running score 0.756944
[mlcrate] Running fold 1, 62697 train samples, 10450 validation samples
[0]	train-auc:0.802126	valid-auc:0.717387
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[1]	train-auc:0.825796	valid-auc:0.729627
[2]	train-auc:0.834618	valid-auc:0.735753
[3]	train-auc:0.839743	valid-auc:0.73949
[4]	train-auc:0.844474	valid-auc:0.742672
[5]	train-auc:0.84877	valid-auc:0.744647
[6]	train-auc:0.85174	valid-auc:0.74648
[7]	train-auc:0.85457	valid-auc:0.74712
[8]	train-auc:0.856687	valid-auc:0.747697
[9]	train-auc:0.859483	valid-auc:0.747975
[10]	train-auc:0.861243	valid-auc:0.748512
[11]	train-auc:0.862815	valid-auc:0.748757
[12]	train-auc:0.863906	valid-auc:0.7492
[13]	train-auc:0.865851	valid-auc:0.749713
[14]	train-auc:0.868062	valid-auc:0.749515
[15]	train-auc:0.869637	valid-auc:0.749912
[16]	train-auc:0.871771	valid-auc:0.75038


[184]	train-auc:0.981349	valid-auc:0.766726
[185]	train-auc:0.981707	valid-auc:0.766507
[186]	train-auc:0.981752	valid-auc:0.766537
[187]	train-auc:0.982009	valid-auc:0.766507
[188]	train-auc:0.982252	valid-auc:0.76647
[189]	train-auc:0.982304	valid-auc:0.766486
[190]	train-auc:0.982685	valid-auc:0.766557
[191]	train-auc:0.982843	valid-auc:0.766549
[192]	train-auc:0.983282	valid-auc:0.766817
[193]	train-auc:0.983736	valid-auc:0.766885
[194]	train-auc:0.984124	valid-auc:0.767017
[195]	train-auc:0.984312	valid-auc:0.767066
[196]	train-auc:0.984448	valid-auc:0.767108
[197]	train-auc:0.984755	valid-auc:0.767043
[198]	train-auc:0.985026	valid-auc:0.766946
[199]	train-auc:0.985181	valid-auc:0.766927
[200]	train-auc:0.98555	valid-auc:0.767132
[201]	train-auc:0.985899	valid-auc:0.766853
[202]	train-auc:0.986258	valid-auc:0.766918
[203]	train-auc:0.986358	valid-auc:0.767029
[204]	train-auc:0.986477	valid-auc:0.767147
[205]	train-auc:0.986769	valid-auc:0.767118
[206]	train-auc:0.98686	valid-auc:

...

[29]	train-auc:0.891283	valid-auc:0.744873
[30]	train-auc:0.89239	valid-auc:0.745274
[31]	train-auc:0.893811	valid-auc:0.745514
[32]	train-auc:0.895208	valid-auc:0.745662
[33]	train-auc:0.896982	valid-auc:0.745815
[34]	train-auc:0.898421	valid-auc:0.745905
[35]	train-auc:0.899745	valid-auc:0.745802
[36]	train-auc:0.900568	valid-auc:0.746091
[37]	train-auc:0.901592	valid-auc:0.746103
[38]	train-auc:0.902811	valid-auc:0.746
[39]	train-auc:0.903576	valid-auc:0.746118
[40]	train-auc:0.904725	valid-auc:0.746165
[41]	train-auc:0.905897	valid-auc:0.745929
[42]	train-auc:0.906881	valid-auc:0.74601
[43]	train-auc:0.907859	valid-auc:0.74638
[44]	train-auc:0.908952	valid-auc:0.746767
[45]	train-auc:0.910242	valid-auc:0.746757
[46]	train-auc:0.911377	valid-auc:0.746896
[47]	train-auc:0.912453	valid-auc:0.746993
[48]	train-auc:0.913246	valid-auc:0.746998
[49]	train-auc:0.913945	valid-auc:0.747215
[50]	train-auc:0.91478	valid-auc:0.747324
[51]	train-auc:0.91537	valid-auc:0.747414
[52]	train-auc:0.91

[218]	train-auc:0.986206	valid-auc:0.755425
[219]	train-auc:0.986335	valid-auc:0.755488
[220]	train-auc:0.9865	valid-auc:0.755709
[221]	train-auc:0.986795	valid-auc:0.75582
[222]	train-auc:0.986925	valid-auc:0.755927
[223]	train-auc:0.986992	valid-auc:0.755884
[224]	train-auc:0.987186	valid-auc:0.755914
[225]	train-auc:0.987406	valid-auc:0.755838
[226]	train-auc:0.987564	valid-auc:0.755825
[227]	train-auc:0.987645	valid-auc:0.755755
[228]	train-auc:0.987736	valid-auc:0.755729
[229]	train-auc:0.987772	valid-auc:0.755722
[230]	train-auc:0.987998	valid-auc:0.755603
[231]	train-auc:0.98818	valid-auc:0.755643
[232]	train-auc:0.988356	valid-auc:0.755662
[233]	train-auc:0.988616	valid-auc:0.755646
[234]	train-auc:0.988878	valid-auc:0.755615
[235]	train-auc:0.988919	valid-auc:0.755643
[236]	train-auc:0.988959	valid-auc:0.755684
[237]	train-auc:0.989035	valid-auc:0.75565
[238]	train-auc:0.989246	valid-auc:0.755693
[239]	train-auc:0.989454	valid-auc:0.755907
[240]	train-auc:0.989541	valid-auc:0.

[406]	train-auc:0.998974	valid-auc:0.757687
[407]	train-auc:0.998986	valid-auc:0.757653
[408]	train-auc:0.999013	valid-auc:0.757718
[409]	train-auc:0.999023	valid-auc:0.757701
[410]	train-auc:0.999038	valid-auc:0.757768
[411]	train-auc:0.999061	valid-auc:0.757799
[412]	train-auc:0.999092	valid-auc:0.757957
[413]	train-auc:0.999107	valid-auc:0.757916
[414]	train-auc:0.999115	valid-auc:0.757928
[415]	train-auc:0.999134	valid-auc:0.75807
[416]	train-auc:0.999136	valid-auc:0.758024
[417]	train-auc:0.99916	valid-auc:0.758144
[418]	train-auc:0.999164	valid-auc:0.758097
[419]	train-auc:0.999177	valid-auc:0.758204
[420]	train-auc:0.999189	valid-auc:0.758182
[421]	train-auc:0.9992	valid-auc:0.758201
[422]	train-auc:0.999212	valid-auc:0.758234
[423]	train-auc:0.999219	valid-auc:0.758199
[424]	train-auc:0.999235	valid-auc:0.758173
[425]	train-auc:0.999249	valid-auc:0.758074
[426]	train-auc:0.999261	valid-auc:0.757994
[427]	train-auc:0.99926	valid-auc:0.757984
[428]	train-auc:0.999271	valid-auc:0.

[115]	train-auc:0.956453	valid-auc:0.763373
[116]	train-auc:0.956807	valid-auc:0.763368
[117]	train-auc:0.957734	valid-auc:0.763518
[118]	train-auc:0.95844	valid-auc:0.763776
[119]	train-auc:0.959171	valid-auc:0.763803
[120]	train-auc:0.95945	valid-auc:0.763899
[121]	train-auc:0.959903	valid-auc:0.764124
[122]	train-auc:0.959947	valid-auc:0.764188
[123]	train-auc:0.960494	valid-auc:0.764189
[124]	train-auc:0.960777	valid-auc:0.764349
[125]	train-auc:0.96101	valid-auc:0.76444
[126]	train-auc:0.961297	valid-auc:0.764368
[127]	train-auc:0.961453	valid-auc:0.764383
[128]	train-auc:0.961883	valid-auc:0.764459
[129]	train-auc:0.962152	valid-auc:0.764631
[130]	train-auc:0.962688	valid-auc:0.764987
[131]	train-auc:0.963039	valid-auc:0.765041
[132]	train-auc:0.963705	valid-auc:0.765131
[133]	train-auc:0.964102	valid-auc:0.765145
[134]	train-auc:0.964596	valid-auc:0.765003
[135]	train-auc:0.965013	valid-auc:0.764907
[136]	train-auc:0.965282	valid-auc:0.765055
[137]	train-auc:0.96537	valid-auc:0.

[303]	train-auc:0.99614	valid-auc:0.768946
[304]	train-auc:0.996183	valid-auc:0.768896
[305]	train-auc:0.996277	valid-auc:0.768771
[306]	train-auc:0.996336	valid-auc:0.768762
[307]	train-auc:0.996372	valid-auc:0.768753
[308]	train-auc:0.996448	valid-auc:0.768863
[309]	train-auc:0.996525	valid-auc:0.768836
[310]	train-auc:0.996642	valid-auc:0.768816
[311]	train-auc:0.996662	valid-auc:0.768817
[312]	train-auc:0.996679	valid-auc:0.768813
[313]	train-auc:0.996767	valid-auc:0.768792
[314]	train-auc:0.996808	valid-auc:0.768795
[315]	train-auc:0.996868	valid-auc:0.768886
[316]	train-auc:0.996923	valid-auc:0.768889
[317]	train-auc:0.996991	valid-auc:0.768867
[318]	train-auc:0.997026	valid-auc:0.768861
[319]	train-auc:0.997061	valid-auc:0.768874
[320]	train-auc:0.997117	valid-auc:0.768934
[321]	train-auc:0.997173	valid-auc:0.768899
[322]	train-auc:0.997191	valid-auc:0.768886
[323]	train-auc:0.997211	valid-auc:0.768854
[324]	train-auc:0.997266	valid-auc:0.768774
[325]	train-auc:0.997341	valid-au

In [35]:
submit = make_submission(p_test_xgb)
submit.to_csv(f'{PATH}\\AV_Stud\\xgb_1_06.csv', index=False)
submit.head()

,id,is_pass
0,1626_45,0.354804
1,11020_130,0.950365
2,12652_146,0.595852
3,7038_72,0.037997
4,888_71,0.162516


In [36]:
np.mean(p_test_xgb), np.std(p_test_xgb)

(0.60203171, 0.25843671)

In [37]:
np.save(f'{PATH}\\AV_Stud\\stack_preds\\xgb_embedding', p_train_xgb)

In [38]:
np.save(f'{PATH}\\AV_Stud\\xgb_train_embedding', x_train_final_dnn)
np.save(f'{PATH}\\AV_Stud\\xgb_test_embedding', x_test_final_dnn)